# How often is X mentioned on Twitter?

#### Load Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import altair as alt
import altair_grid as altgrid

In [3]:
alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

---

## Get data

#### Counts pulled with the Twarc library. [Check it out](https://twarc-project.readthedocs.io/en/latest/twarc2_en_us/). 

In [35]:
# !twarc2 counts --granularity day --csv --archive 'elonmusk OR Elon Musk' data/raw/@elonmusk_mention_counts.csv

---

## Read data

#### @elonmusk or 'Elon Musk'

In [36]:
mentioned = "elonmusk"

src = pd.read_csv(
    "data/raw/@elonmusk_mention_counts.csv", parse_dates=["start", "end"]
).sort_values("start", ascending=False)

#### Process dates

In [37]:
src["year"] = pd.to_datetime(src["start"]).dt.strftime("%Y")
src["month_year"] = pd.to_datetime(src["start"]).dt.strftime("%Y-%m")
src["date"] = pd.to_datetime(src["start"]).dt.strftime("%Y-%m-%d")

In [38]:
src[src["date"] == src["date"].max()]

,start,end,day_count,year,month_year,date
30,2022-05-01 00:00:00+00:00,2022-05-01 15:31:53+00:00,576105,2022,2022-05,2022-05-01


In [39]:
src_grouped = (
    src.groupby(["date", "year", "month_year"]).agg({"day_count": sum}).reset_index()
)

#### Slim down and re-order the dataframe

In [40]:
df = (
    src_grouped[["year", "month_year", "date", "day_count"]]
    .sort_values("date", ascending=False)
    .copy()
)

In [41]:
df.head()

,year,month_year,date,day_count
5885,2022,2022-05,2022-05-01,576105
5884,2022,2022-04,2022-04-30,1291524
5883,2022,2022-04,2022-04-29,1799211
5882,2022,2022-04,2022-04-28,3330786
5881,2022,2022-04,2022-04-27,3310309


#### When was the first mention? 

In [42]:
first = df[df["day_count"] > 0]["date"].tail(1).iloc[0]
first

'2007-03-21'

#### How many total mentions? 

In [43]:
df.day_count.sum()

187315355

#### Average mentions? 

In [44]:
df.day_count.mean()

31823.8795446823

#### Make a new dataframe starting from first mention

In [45]:
df_complete = df[df["date"] >= first]

#### Which day was mentioned the most? 

In [46]:
df_complete[df_complete["day_count"] == df_complete["day_count"].max()]

,year,month_year,date,day_count
5880,2022,2022-04,2022-04-26,5490486


In [47]:
df_complete.sort_values("day_count", ascending=False).head(10)

,year,month_year,date,day_count
5880,2022,2022-04,2022-04-26,5490486
5879,2022,2022-04,2022-04-25,4096949
5882,2022,2022-04,2022-04-28,3330786
5881,2022,2022-04,2022-04-27,3310309
5868,2022,2022-04,2022-04-14,2052518
5869,2022,2022-04,2022-04-15,1925252
5883,2022,2022-04,2022-04-29,1799211
5445,2021,2021-02,2021-02-15,1291985
5884,2022,2022-04,2022-04-30,1291524
5859,2022,2022-04,2022-04-05,1287689


#### Chart it

In [48]:
alt.Chart(df_complete.head(365 * 10)).mark_area().encode(
    x=alt.X("date:T", axis=alt.Axis(format="%b. %Y", tickCount=5), title=""),
    y=alt.Y("day_count:Q", title="", axis=alt.Axis(tickCount=5, format="0,M")),
).properties(width=900, title="Daily mentions of Elon Musk on Twitter")

alt.Chart(...)

---

## Aggregate 

#### Groupby month/year

In [49]:
months = (
    df_complete.groupby(["month_year"])
    .agg({"day_count": sum})
    .sort_values("month_year", ascending=False)
    .reset_index()
)

#### Chart it

In [51]:
alt.Chart(months.head(120)).mark_area().encode(
    x=alt.X("month_year:T", axis=alt.Axis(format="%b. %Y", tickCount=5), title=""),
    y=alt.Y("day_count:Q", title=" ", axis=alt.Axis(tickCount=5, format="0,M")),
).properties(width=900, title="Monthly mentions of Elon Musk on Twitter")

alt.Chart(...)

## Exports

In [57]:
months[months["month_year"] > "2014-12"].to_csv(
    f"data/processed/twitter_mentions_{mentioned}_months.csv", index=False
)
df_complete[df_complete["date"] > "2014-12-31"].to_csv(
    f"data/processed/twitter_mentions_{mentioned}_days.csv", index=False
)

In [58]:
months

,month_year,day_count
0,2022-05,576105
1,2022-04,35458535
2,2022-03,8063725
3,2022-02,5765964
4,2022-01,7330772
5,2021-12,7569631
6,2021-11,6561856
7,2021-10,5528700
8,2021-09,3247265
9,2021-08,4131464


In [59]:
months[months["month_year"] > "2021-01"].day_count.mean()

7676679.0